In [1]:
# Import packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Hide all warnings
import warnings
warnings.filterwarnings("ignore")

## Import and Concatenate Original Event Datasets

In [2]:
# Read the data from the CSV file
data1 = pd.read_csv("../Raw_Data/socialinsider_events_2024-05.csv")
data2 = pd.read_csv("../Raw_Data/socialinsider_events_2024-06.csv")
data3 = pd.read_csv("../Raw_Data/socialinsider_events_2024-07.csv")
data4 = pd.read_csv("../Raw_Data/socialinsider_events_2024-08.csv")
data5 = pd.read_csv("../Raw_Data/socialinsider_events_2024-09.csv")
data6 = pd.read_csv("../Raw_Data/socialinsider_events_2024-1008.csv")
data7 = pd.read_csv("../Raw_Data/socialinsider_events_2024-1029.csv")

In [3]:
# Concatenate the data
data = pd.concat([data1, data2, data3, data4, data5, data6, data7])

# Sort the data by time_created
data = data.sort_values(by="time_created")

## Data Transformation

Create a column to determine whether the event is from converted users

In [4]:
# Create a column that show whether the event is successful or not
buy_users = data[
    (data["event_name"] == "New Client") | (data["event_name"] == "Buy Success")
]["user_id"].tolist()
# 1 if the user is in buy_users, 0 otherwise
data["successful"] = data["user_id"].apply(lambda x: 1 if x in buy_users else 0)

Convert the timestamp

In [5]:
# create the date that the event was created
data["Date_Created"] = pd.to_datetime(data["time_created"]).dt.date
# create the hour that the event was created
data["Hour_Created"] = pd.to_datetime(data["time_created"]).dt.hour
# create the day of the week that the event was created
data["Day_Of_Week_Created"] = pd.to_datetime(data["time_created"]).dt.day_name()

### User Data Transformation

Success, event count, and country

In [6]:
data

event_name  \
237525        profile load success   
237524              section change   
237523                     compare   
237522              section change   
237521               profile visit   
...                            ...   
4              i10e-section-select   
3              i10e-section-select   
2              i10e-section-select   
1              posts section visit   
0       posts section load success   

                                                  user_id  \
237525  38bd4c351e290c6024d39d44b9a2750ea666eefc9b1069...   
237524  38bd4c351e290c6024d39d44b9a2750ea666eefc9b1069...   
237523  38bd4c351e290c6024d39d44b9a2750ea666eefc9b1069...   
237522  38bd4c351e290c6024d39d44b9a2750ea666eefc9b1069...   
237521  38bd4c351e290c6024d39d44b9a2750ea666eefc9b1069...   
...                                                   ...   
4       972a3338a20a8ea71a6dec6011185db6fe6d6026cf74dd...   
3       972a3338a20a8ea71a6dec6011185db6fe6d6026cf74dd...   
2       972a3338a20a8ea71a6dec6011185db6fe6d6026cf74dd...   
1       972a3338a20a8ea71a6dec6011185db6fe6d6026cf74dd...   
0       972a3338a20a8ea71a6dec6011185db6fe6d6026cf74dd...   

                    time_created user_type       time_zone       country  \
237525  2024-04-30T20:59:17.702Z     trial   Europe/London  Britain (UK)   
237524  2024-04-30T21:00:06.728Z     trial   Europe/London  Britain (UK)   
237523  2024-04-30T21:00:06.765Z     trial   Europe/London  Britain (UK)   
237522  2024-04-30T21:00:20.243Z     trial   Europe/London  Britain (UK)   
237521  2024-04-30T21:00:27.275Z     trial   Europe/London  Britain (UK)   
...                          ...       ...             ...           ...   
4       2024-10-29T14:54:45.795Z     trial  America/Bogota      Colombia   
3       2024-10-29T14:54:47.209Z     trial  America/Bogota      Colombia   
2       2024-10-29T14:54:49.148Z     trial  America/Bogota      Colombia   
1       2024-10-29T14:54:49.168Z     trial  America/Bogota      Colombia   
0       2024-10-29T14:54:49.475Z     trial  America/Bogota      Colombia   

           view   platform report_type  load_time  successful Date_Created  \
237525  profile         ig         NaN      1.264           0   2024-04-30   
237524  profile         ig         NaN        NaN           0   2024-04-30   
237523  profile  instagram         NaN        NaN           0   2024-04-30   
237522  profile         ig         NaN        NaN           0   2024-04-30   
237521  profile         tw         NaN        NaN           0   2024-04-30   
...         ...        ...         ...        ...         ...          ...   
4       hashtag        xch         NaN        NaN           0   2024-10-29   
3       hashtag        xch         NaN        NaN           0   2024-10-29   
2       hashtag        xch         NaN        NaN           0   2024-10-29   
1       hashtag        xch         NaN        NaN           0   2024-10-29   
0       hashtag        xch         NaN      0.309           0   2024-10-29   

        Hour_Created Day_Of_Week_Created  
237525            20             Tuesday  
237524            21             Tuesday  
237523            21             Tuesday  
237522            21             Tuesday  
237521            21             Tuesday  
...              ...                 ...  
4                 14             Tuesday  
3                 14             Tuesday  
2                 14             Tuesday  
1                 14             Tuesday  
0                 14             Tuesday  

[1286351 rows x 14 columns]

In [7]:
user_data = (
    data.groupby("user_id")
    .agg(
        {
            "event_name": "count",  # Count events
            "country": "first",  # Take the first occurrence of country (assuming it's the same for each user)
        }
    )
    .reset_index()
    .rename(columns={"event_name": "event_count"})  # Rename for clarity
    .sort_values(by="event_count", ascending=False)  # Sort by event_count
)

In [8]:
# Convert counties to separate columns if they are in certain countries
user_data["country_United_States"] = user_data["country"].apply(
    lambda x: 1 if x == "United States" else 0
)
user_data["country_Saudi_Arabia"] = user_data["country"].apply(
    lambda x: 1 if x == "Saudi Arabia" else 0
)
user_data["country_India"] = user_data["country"].apply(
    lambda x: 1 if x == "India" else 0
)
user_data["country_Britain"] = user_data["country"].apply(
    lambda x: 1 if x == "Britain (UK)" else 0
)
user_data["country_Italy"] = user_data["country"].apply(
    lambda x: 1 if x == "Italy" else 0
)

In [9]:
# Add the column to show whether the user is successful or not
user_data["successful"] = user_data["user_id"].apply(
    lambda x: 1 if x in buy_users else 0
)

In [10]:
user_data


user_id  event_count  \
11103  cd6d41c28b017733e39fc13e4d4ae3b308c6baff52561d...         6973   
4820   5929c3eec5a2b12ee4aa911b8093b33ac7b798d59e3b14...         4700   
5297   622748abdc404cbf098568074c232b04762d574fa48130...         3925   
13620  fd15147418cba03da1908de36ac79bc24589c9d1474716...         3538   
5004   5c54eb7d3e4821d8e633e7ef23dacafe3f5a6530715510...         3387   
...                                                  ...          ...   
4773   58620f4bb811ae3d3194ec268accb68794b16d33537508...            0   
13168  f4f9987e642659097752e755b740f7e487f6c252f5be0c...            0   
5361   63432b4995d7737fe96541f5d19bf5552ae9a8623ad104...            0   
2341   2a6612b86399adeeb92d2b44603d8736b1dbfcb84ccf24...            0   
5710   69d1d96d2d29467d3eddb1bbaec5ed364941c09629d401...            0   

             country  country_United_States  country_Saudi_Arabia  \
11103  United States                      1                     0   
4820       Indonesia                      0                     0   
5297         Ukraine                      0                     0   
13620       Thailand                      0                     0   
5004         Germany                      0                     0   
...              ...                    ...                   ...   
4773            None                      0                     0   
13168           None                      0                     0   
5361            None                      0                     0   
2341            None                      0                     0   
5710            None                      0                     0   

       country_India  country_Britain  country_Italy  successful  
11103              0                0              0           0  
4820               0                0              0           0  
5297               0                0              0           0  
13620              0                0              0           0  
5004               0                0              0           0  
...              ...              ...            ...         ...  
4773               0                0              0           0  
13168              0                0              0           0  
5361               0                0              0           0  
2341               0                0              0           0  
5710               0                0              0           0  

[13780 rows x 9 columns]

Transform Load Time to maximum load time and average load time

In [11]:
# get the average load time for each client and sort by the average load time
average_load_time = data.groupby("user_id")["load_time"].mean().sort_values()

# add the column to show the average load time for each user
user_data["average_load_time"] = user_data["user_id"].apply(
    lambda x: average_load_time[x] if x in average_load_time else None
)
# Replace missing values in 'average_load_time' with the median
median_load_time = user_data["average_load_time"].median()
user_data["average_load_time"].fillna(median_load_time, inplace=True)

In [12]:
# get the maximum load time for each client and sort by the maximum load time
max_load_time = data.groupby("user_id")["load_time"].max().sort_values()

# add the column to show the maximum load time for each user
user_data["max_load_time"] = user_data["user_id"].apply(
    lambda x: max_load_time[x] if x in max_load_time else None
)
# Replace missing values in 'max_load_time' with the median
median_max_load_time = user_data["max_load_time"].median()
user_data["max_load_time"].fillna(median_max_load_time, inplace=True)

Transform events

In [13]:
event_counts = (
    data.groupby("user_id")
    .apply(
        lambda x: pd.Series(
            {
                "event_bench_load_success_count": (
                    x["event_name"] == "bench load success"
                ).sum(),
                "event_profile_search_success_count": (
                    x["event_name"] == "profile search success"
                ).sum(),
                "event_add_profile_success_count": (
                    x["event_name"] == "add profile success"
                ).sum(),
                "event_upgrade_plan_count": (x["event_name"] == "upgrade_plan").sum(),
                "event_pricing_model_count": (
                    x["event_name"] == "pricing modal visited"
                ).sum(),
                "event_profile_load_fail_count": (
                    x["event_name"] == "profile load fail"
                ).sum(),
                "event_email_receipt_count": (x["event_name"] == "email receipt").sum(),
            }
        )
    )
    .reset_index()
)

# join the event_counts dataframe on user_data using user_id
user_data = user_data.merge(event_counts, on="user_id", how="left")

Transform Platform

In [14]:
# convert to shortcut
data["platform"] = data["platform"].apply(
    lambda x: (
        "fb"
        if x == "facebook" or x == "showFacebook"
        else (
            "tw"
            if x == "twitter"
            else (
                "ig"
                if x == "instagram"
                else (
                    "yt"
                    if x == "youtube"
                    else (
                        "li"
                        if x == "linkedin"
                        else (
                            "tk"
                            if x == "tiktok"
                            else "xch" if x == "cross-platform" else x
                        )
                    )
                )
            )
        )
    )
)

# rename to platform + original name + count
data["platform"] = data["platform"].apply(lambda x: f"platform_{x}_count")

# count
platform_data = (
    data.groupby(["user_id", "platform"]).size().unstack(fill_value=0).reset_index()
)

# merge
user_data = user_data.merge(platform_data, on="user_id", how="left")

# add a column in user_data for total platforms of each user
user_data["platform_total_count"] = user_data.iloc[:, 5:].sum(axis=1)

Transform View

In [15]:
# rename
data["view"] = data["view"].apply(lambda x: f"view_{x}")

# convert
view_data = data.groupby(["user_id", "view"]).size().unstack(fill_value=0).reset_index()

# merge
user_data = user_data.merge(view_data, on="user_id", how="left")

#### Filter Out Data that only has 1 event

In [16]:
# filter out users with event count <= 1
user_data = user_data[user_data["event_count"] > 1]

In [17]:
user_data

user_id  event_count  \
0      cd6d41c28b017733e39fc13e4d4ae3b308c6baff52561d...         6973   
1      5929c3eec5a2b12ee4aa911b8093b33ac7b798d59e3b14...         4700   
2      622748abdc404cbf098568074c232b04762d574fa48130...         3925   
3      fd15147418cba03da1908de36ac79bc24589c9d1474716...         3538   
4      5c54eb7d3e4821d8e633e7ef23dacafe3f5a6530715510...         3387   
...                                                  ...          ...   
13172  a17f2604cd4bfda6d2fa30d7c1bcee33f878e7a9d751ee...            2   
13173  a108f54169f8630e53c67de9861f998666de1aaf78d571...            2   
13174  a100308eeac58e2a066a43e0f3a4aa20bcff2706c872f0...            2   
13175  44467b18ea5aedb01bfbb16459600fc85fa26823f29cd7...            2   
13176  44c20979a2afb549cd7e24d61df676d2db4f09e25aca30...            2   

             country  country_United_States  country_Saudi_Arabia  \
0      United States                      1                     0   
1          Indonesia                      0                     0   
2            Ukraine                      0                     0   
3           Thailand                      0                     0   
4            Germany                      0                     0   
...              ...                    ...                   ...   
13172        Morocco                      0                     0   
13173        Nigeria                      0                     0   
13174   Saudi Arabia                      0                     1   
13175  United States                      1                     0   
13176          Egypt                      0                     0   

       country_India  country_Britain  country_Italy  successful  \
0                  0                0              0           0   
1                  0                0              0           0   
2                  0                0              0           0   
3                  0                0              0           0   
4                  0                0              0           0   
...              ...              ...            ...         ...   
13172              0                0              0           0   
13173              0                0              0           0   
13174              0                0              0           0   
13175              0                0              0           0   
13176              0                0              0           0   

       average_load_time  ...  view_nan  view_page  view_postsfeed  \
0               1.238146  ...        16          0               8   
1               1.040046  ...        19          0               0   
2               1.823422  ...        22          0               0   
3               1.692242  ...        46          0               8   
4               1.908071  ...       126          0               0   
...                  ...  ...       ...        ...             ...   
13172           1.741000  ...         2          0               0   
13173           1.741000  ...         2          0               0   
13174           1.741000  ...         4          0               0   
13175           1.741000  ...         2          0               0   
13176           1.741000  ...         2          0               0   

       view_profile  view_proj  view_projecthome  view_reports  view_search  \
0              6945          0                 2             0            0   
1              4674          0                 3             0            0   
2              3889          0                11             0            0   
3              3430          0                38             3            0   
4              3216          0                14             0            0   
...             ...        ...               ...           ...          ...   
13172             0          0                 0             0            0   
13173             0          0                 0     

## Train Test Split

In [18]:
user_data_train = user_data.drop(["user_id", "country"], axis=1)

In [19]:
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame `df` with features `X` and a target column `y`
X = user_data_train.drop("successful", axis=1)
y = user_data_train[["successful"]]

# Perform train-test split, with 80% of the data used for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10
)

# Now you have your train and test datasets ready
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

ModuleNotFoundError: No module named 'sklearn'

# Resampling

- Experiment with Downsampling and Oversampling methods on Random Forest & Gradient Boosting Model (selected due to respectively high preformance compared to other baseline models)
- Combination of Downsampling and Oversampling is used to generate the final resampled dataset for modeling
    - Different Thresholds are tested on RF and GB 

In [ ]:
# import packages
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix,
    f1_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import resample
from collections import Counter
from sklearn.model_selection import GridSearchCV
from itertools import product


### Oversampling + Downsampling

In [ ]:
# Separate majority and minority classes
majority_class = X_train[y_train['successful'] == 0]
minority_class = X_train[y_train['successful'] == 1]
majority_labels = y_train[y_train['successful'] == 0]
minority_labels = y_train[y_train['successful'] == 1]

In [ ]:
# print length of majority and minority classes
print('Majority Class Length: ', len(majority_class))
print('Minority Class Length: ', len(minority_class))

### Resampled Data

#### According to best & reasonable recall

In [ ]:
# Resample and save data with 1000 size majority and 530 size minority
minority_upsampled = resample(
    minority_class,
    replace=True,
    n_samples=534,
    random_state=42
)
minority_labels_upsampled = resample(
    minority_labels,
    replace=True,
    n_samples=534,
    random_state=42
)

majority_downsampled = resample(
    majority_class,
    replace=False,
    n_samples=1000,
    random_state=42
)
majority_labels_downsampled = resample(
    majority_labels,
    replace=False,
    n_samples=1000,
    random_state=42
)

X_train_resampled = pd.concat([majority_downsampled, minority_upsampled]).reset_index(drop=True)
y_train_resampled = pd.concat([majority_labels_downsampled, minority_labels_upsampled]).reset_index(drop=True)

In [ ]:
# Save the resampled data
X_train_resampled.to_csv("../Final_Data/X_train.csv", index=False)
y_train_resampled.to_csv("../Final_Data/y_train.csv", index=False)